<h3><center>Using RSpec with Factory Girl, Shoulda Matcher (include Devise Token Auth)</center></h3>

<b>Rspec:</b> is a 'Domain Specific Language' (DSL) testing tool written in Ruby to test Ruby code. It is a behavior-driven development (BDD) framework which is extensively used in the production applications.

In [ ]:
#INSTALLATION
#gemfile
gem "rspec"


#create rspec
rails generate rspec:install

#model testing
rails generate rspec:model <model name>
  
#controller testing
rails generate rspec:controller <conller name>

 <b>Note</b> We will combine RSpec with Factory Girl and Shoulda Matcher so we just skip rspec basic.

<b>Factory girl rails</b>: is a fixtures replacement with a straightforward definition syntax, support for multiple build strategies (saved instances, unsaved instances, attribute hashes, and stubbed objects), and support for multiple factories for the same class (user, admin_user, and so on), including factory inheritance.
    ==> easy for building object

In [ ]:
#INSTALLATION
#gem file
gem 'factory_girl_rails', '~> 4.9'

#spec/rails_helper.rb
config.include FactoryGirl::Syntax::Methods

<b>Note</b>: when we create rspec testing case, factory girl will automatically generate file in spec/factories

In [ ]:
#For example

#spec/factories/books.rb
FactoryGirl.define do
  factory :book do
    title "this is spatar"
    content "nah"
  end
end

<b>Shoulda Matcher</b>: this gem will ease the writing test case things with many helpers.

In [ ]:
#INSTALLATION

#gemfile
gem 'shoulda-matchers', git: 'https://github.com/thoughtbot/shoulda-matchers.git', branch: 'rails-5'

#spec/rails_helper.rb
Shoulda::Matchers.configure do |config|
    config.integrate do |with|
      # Choose a test framework:
      with.test_framework :rspec
      with.library :rails
    end
  end

<h3><center>UNIT TEST - MODEL TESTING (association, validation, ...)</center></h3>

In [ ]:
#in spec/models/book_spec.rb

require 'rails_helper'

RSpec.describe Book, type: :model do
  descrbibe 'it should validate association' do
    it{should_belong_to(:type)}
    it{should_belong_to(:user)}
  end
  
  describe 'it should validate presence of title' do
    it{should_validate_prensence_of(:title)}
  end
end


#in spec/models/type_spec.rb
require 'rails_helper'
RSpec.describe Type, type: :model do
  describe 'it should validate association' do
    it{should_have_many(:books)}
  end
  
  describe 'it should validate type presence' do
    it{should_validate_presence_of(:title)}
  end
end


# in spec/models/user_spec.rb
require 'rails_helper'
RSpec.describe User, type: :model do
  describe 'it should validate association' do
    it {should_have_many(:books)}
  end
end

<h3><center>CONTROLLER TESTING (routes testing, response testing)</center></h3>

In [ ]:
#in spec/controllers/book_controller_spec.rb

RSpec.describe Api::V1::BooksController, type: :controller do

  describe 'route testing' do
    it {should route(:get, '/api/v1/books').to(action: :index)}
    it {should route(:get, '/api/v1/books/1').to(action: :show, id: 1)}
    it {should route(:post, '/api/v1/books').to(action: :create)}
    it {should route(:put, '/api/v1/books/1').to(action: :update, id: 1)}
    it {should route(:delete, '/api/v1/books/1').to(action: :destroy, id: 1)}
    it {should route(:delete, '/api/v1/delete_all').to(action: :delete_all)}
  end

  
  #testing response
  context 'Testing actions in controller with respond with' do
    describe 'GET #index' do
      before {get :index}
      it {should respond_with(200)}
    end

    describe 'GET #show' do
      before do
        book = FactoryGirl.create(:book)
        get :show, params: {id: book.id}
      end
      it {
        should respond_with(200)
      }
    end

    describe 'POST #create' do
      let(:valid_params) do
        {
            title: "this is spatar",
            content: "stupid book",
            type_id: 1,
            user_id: 1
        }
      end

      before do
        #this is for devise_token_auth
        user = FactoryGirl.build(:user)
        auth_headers = user.create_new_auth_token
        request.headers.merge!(auth_headers)
        post :create, params: valid_params
      end
      it {
        should respond_with(200)
      }
    end

    describe 'PUT #update' do
      let(:valid_params) do
        {
            book: {
                title: "this is spatar",
                content: "stupid book",
                type_id: 1,
                user_id: 1
            }
        }
      end

      before do
        #book = FactoryGirl.create(:book)
        user = FactoryGirl.create(:user)
        auth_headers = user.create_new_auth_token
        request.headers.merge!(auth_headers)
        put :update, params: {id: 298486374, book: valid_params}
      end
      it {should respond_with(200)}
    end


    describe 'DELETE #destroy' do
      before{
        user = FactoryGirl.create(:user)
        auth_headers = user.create_new_auth_token
        request.headers.merge!(auth_headers)
        delete :destroy, params: {id: 298486374}
      }
      it {should respond_with(200)}
    end

  end

end
